In [ ]:
#Question-1
from keras import datasets,Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D,BatchNormalization,Dropout
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam


(xtr,ytr),(xt,yt)=datasets.cifar100.load_data()
ytr=ytr.reshape(-1,)
yt=yt.reshape(-1,)
plt.figure(figsize=(10,10))
for image in range(0,20):
  i=image
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  j=i+0
  data_plot=xtr[j]
  plt.imshow(data_plot)
  plt.xlabel(str(yt[j]))
plt.show()

xtr=xtr/255
xt=xt/255
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(xtr)
model=Sequential()
model.add(Conv2D(input_shape=(32,32,3),kernel_size=(2,2),padding='same',strides=(2,2),filters=3))
model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same'))

model.add(Conv2D(kernel_size=(2,2),padding='same',strides=(2,2),filters=64))
model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same'))

model.add(Conv2D(kernel_size=(2,2),padding='same',strides=(2,2),filters=64))
model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same'))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(100,activation='softmax'))

model.summary()
opt=Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(xtr,ytr,epochs=70,validation_data=(xt,yt))

test_loss,test_acc=model.evaluate(xt,yt)
print("test accuracy:",test_acc)
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
#question-2
import tensorflow as tf
from keras import datasets,Sequential
from keras.applications import VGG16
from keras.layers import Dense,Flatten,Input,Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

(xtr,ytr),(xt,yt)=datasets.cifar10.load_data()
print(xtr.shape)
xtr=xtr/255.0
xt=xt/255.0
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(xtr);
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in vgg16_base.layers:
    layer.trainable = False
inputs=Input(shape=(32,32,3))
x=vgg16_base(inputs,training=False)
x=Flatten()(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.1)(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.1)(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history1=model.fit(xtr, ytr, epochs=50, validation_data=(xt, yt),batch_size=64)

# Evaluate the model
test_loss, test_acc = model.evaluate(xt ,yt)
print(f'\nTest accuracy: {test_acc}')
print("fine tune")
vgg16_base.trainable=True
print(len(vgg16_base.layers))
fine_tune_at=8
for layer in vgg16_base.layers[:fine_tune_at]:
  layer.trainable=False

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history2=model.fit(xtr, ytr, epochs=50, validation_data=(xt, yt),batch_size=64)

# Evaluate the model
test_loss, test_acc = model.evaluate(xt ,yt)
print(f'\nTest accuracy: {test_acc}')

plt.plot(history1.history['accuracy'],color='red',label='train')
plt.plot(history1.history['val_accuracy'],color='blue',label='validation')
plt.plot(history2.history['accuracy'],color='red',label='train_finetune')
plt.plot(history2.history['val_accuracy'],color='blue',label='validation_finetune')
plt.legend()
plt.show()
plt.plot(history1.history['loss'],color='red',label='loss_train')
plt.plot(history1.history['val_loss'],color='blue',label='loss_validation')
plt.plot(history2.history['loss'],color='red',label='loss_train_finetune')
plt.plot(history2.history['val_loss'],color='blue',label='loss_validation_finetune')
plt.legend()
plt.show()

In [ ]:
#question3
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/chest_xray/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)
valid_ds=keras.utils.image_dataset_from_directory(
    directory='/content/chest_xray/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)
def process(image,label):
  image=tf.cast(image/255,tf.float32)
  return image,label
train_ds=train_ds.map(process)
valid_ds=valid_ds.map(process)


model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=20,validation_data=valid_ds)
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
#question4
import tensorflow as tf
from keras import Sequential,datasets
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.optimizers import Adam


(xtr,ytr),(xt,yt)=fashion_mnist.load_data()
xtr=xtr/255.0
xt=xt/255.0


model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history1=model.fit(xtr,ytr,epochs=20,validation_data=(xt,yt),validation_split=0.2, batch_size = 512)
test_loss,test_acc=model.evaluate(xt,yt)
print(test_acc)

model1=Sequential()
model1.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',input_shape=(28,28,1)))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))


model1.add(Flatten())
model1.add(Dense(128,activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64,activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(10,activation='softmax'))
model1.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history2=model.fit(xtr,ytr,epochs=20,validation_data=(xt,yt),validation_split=0.2, batch_size = 512)
test_loss1,test_acc1=model.evaluate(xt,yt)
print(test_acc1)

model2=Sequential()
model2.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu',input_shape=(28,28,1)))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model2.add(MaxPool2D(pool_size=(2,2)))


model2.add(Flatten())
model2.add(Dense(256,activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(64,activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(10,activation='softmax'))

model2.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history3=model.fit(xtr,ytr,epochs=20,validation_data=(xt,yt),validation_split=0.2, batch_size = 512)
test_loss2,test_acc2=model.evaluate(xt,yt)
print(test_acc2)
import matplotlib.pyplot as plt
plt.plot(history1.history['loss'],label='model1 train loss')
plt.plot(history1.history['val_loss'],label='model 1 validation loss')

plt.plot(history2.history['loss'],label='model2 train loss')
plt.plot(history2.history['val_loss'],label='model 2 validation loss')

plt.plot(history3.history['loss'],label='model3 train loss')
plt.plot(history3.history['val_loss'],label='model 3 validation loss')

plt.legend()
plt.show()